
# Reconhecimento Facial — 4 Pessoas (Você + 3 colegas)

Este notebook cria um pipeline completo de **reconhecimento facial** para 4 pessoas usando:

- **Detecção e alinhamento:** `MTCNN` (do pacote `facenet-pytorch`)
- **Embeddings faciais:** `InceptionResnetV1` (FaceNet pré‑treinado em VGGFace2, via `facenet-pytorch`)
- **Classificador:** `SVM` (RBF) do `scikit-learn` sobre os embeddings

Por que assim?
- Para datasets pequenos, **extrair embeddings pré‑treinados** e treinar um **classificador leve** costuma dar
  resultados muito melhores e treina bem mais rápido do que treinar uma CNN do zero.
- Se preferir, há também uma **Opção B (Transfer Learning com Keras/MobileNetV2)** no final.

> **Como organizar seus dados** (recomendado):
>
> ```text
> dados_brutos/
> ├── pessoa1_matheus/
> │   ├── img001.jpg
> │   ├── img002.jpg
> │   └── ...
> ├── pessoa2_colegaA/
> ├── pessoa3_colegaB/
> └── pessoa4_colegaC/
> ```
>
> Ideal ter **50+ fotos por pessoa** cobrindo variações de luz, pose e expressão.
> O notebook vai **detectar/alinha** as faces e salvar em `dados_processados/`.
>
> **Observação:** Este notebook foi gerado tomando como base o seu arquivo `"[ Aula 17 ] - CKP04.ipynb"`.
> Você pode copiar/colar células deste novo notebook para o seu, ou rodar este diretamente.



## 1) Instalação de dependências (rode apenas se precisar)

Se der erro de import, rode este bloco. Caso já tenha tudo instalado, pode pular.


In [ ]:

#Se precisar, descomente as linhas abaixo
!pip install --upgrade pip
!pip install facenet-pytorch==2.5.3 mtcnn==0.1.1 opencv-python==4.10.0.84 scikit-learn==1.5.1 joblib==1.4.2
!pip install matplotlib==3.9.0 pandas==2.2.2 numpy==1.26.4
#Para a Opção B (Keras/MobileNetV2):
!pip install tensorflow==2.15.0


  Using cached mtcnn-0.1.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached scikit_learn-1.5.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached mtcnn-0.1.1-py3-none-any.whl (2.3 MB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)
Using cached scikit_learn-1.5.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Successfully uninstalled opencv-python-4.12.0.88
  Attempting uninstall: joblib
    Found existing installation: joblib 1.5.1
    Uninstalling joblib-1.5.1:
      Successfully uninstalled joblib-1.5.1


  Using cached matplotlib-3.9.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached matplotlib-3.9.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 127.1 MB/s  0:00:00
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: pandas
    Found existing installation: pandas 2.3.1
    Uninstalling pandas-2.3.1:
      Successfully uninstalled pandas-2.3.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.10.5
    Uninstalling matplotlib-3.10.5:
      Successfully uninstalled matplotlib-3.10.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3

ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0



## 2) Imports e configuração


In [ ]:
import sys
!{sys.executable} -m pip install --upgrade \
    numpy \
    pandas \
    matplotlib \
    scikit-learn \
    joblib \
    opencv-python \
    facenet-pytorch \
    mtcnn \
    pillow \
    pillow-heif \
    torch torchvision torchaudio \
    tensorflow


  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached matplotlib-3.10.5-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached scikit_learn-1.7.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached mtcnn-1.0.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached numpy-2.2.6-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
Using cac

In [ ]:

import os
from pathlib import Path
import random
import numpy as np
import pandas as pd
import cv2
import joblib
import matplotlib.pyplot as plt

# Torch / facenet-pytorch
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Reprodutibilidade
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Se tiver GPU disponível, ótimo!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device


device(type='cpu')


## 3) Defina os caminhos das pastas

- `dados_brutos/`: suas fotos originais (múltiplas por pessoa, em subpastas).
- `dados_processados/`: faces detectadas/alinhadas que o notebook vai gerar.


In [ ]:
# Verificação das 4 pastas e contagem de imagens por pessoa em dados_brutos/
from pathlib import Path
from collections import Counter
import shutil

VALID_EXT = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".heic"}

root = Path("dados_brutos")

# Corrige casos de zip aninhado: dados_brutos/dados_brutos/...
if (root / "dados_brutos").exists():
    root = root / "dados_brutos"

# Remove pasta extra do macOS, se existir
if (root / "__MACOSX").exists():
    shutil.rmtree(root / "__MACOSX", ignore_errors=True)

def contar_imagens_por_pessoa(pasta_root: Path):
    contagem = {}
    subpastas = [p for p in pasta_root.iterdir() if p.is_dir() and not p.name.startswith(".")]
    for pessoa_dir in sorted(subpastas):
        n = 0
        for f in pessoa_dir.rglob("*"):
            if f.is_file() and f.suffix.lower() in VALID_EXT:
                n += 1
        contagem[pessoa_dir.name] = n
    return contagem

contagem = contar_imagens_por_pessoa(root)

total_arquivos = sum(contagem.values())
classes = sorted(contagem.keys())

print(f"Raiz analisada: {root.resolve()}")
print(f"Total de classes encontradas: {len(classes)} -> {classes}")
print(f"Total de arquivos válidos: {total_arquivos}\n")

print("Contagem por classe:")
for k in classes:
    print(f"- {k}: {contagem[k]} imagens")

faltando = [k for k,v in contagem.items() if v == 0]
if faltando:
    print("\nAtenção: as classes abaixo estão vazias (0 imagens válidas):")
    for k in faltando:
        print(f"- {k}")

# Se quiser exigir exatamente 4 classes, descomente:
# if len(classes) != 4:
#     raise ValueError(f"Esperadas 4 classes, mas encontrei {len(classes)}. Verifique nomes/pastas.")


FileNotFoundError: [Errno 2] No such file or directory: 'dados_brutos'

In [ ]:

BASE = Path().resolve()

# Ajuste estes nomes se quiser
PASTA_BRUTOS = BASE / "dados_brutos"
PASTA_PROC   = BASE / "dados_processados"

PASTA_PROC.mkdir(exist_ok=True)

print("Brutos:", PASTA_BRUTOS)
print("Processados:", PASTA_PROC)


Brutos: /content/dados_brutos
Processados: /content/dados_processados



## 4) Detecção e alinhamento das faces (MTCNN)

Este passo percorre `dados_brutos/`, detecta/alinha a face principal e salva faces 160x160 em `dados_processados/`,
mantendo a **mesma estrutura de pastas por pessoa**.


In [ ]:

# Detector (ajuste thresholds se estiver perdendo/pegando faces demais)
mtcnn = MTCNN(image_size=160, margin=20, post_process=True, device=device)

def processar_pasta_de_faces(pasta_origem, pasta_destino):
    pasta_origem = Path(pasta_origem)
    pasta_destino = Path(pasta_destino)
    pasta_destino.mkdir(exist_ok=True, parents=True)

    total_in = 0
    total_ok = 0

    for pessoa_dir in pasta_origem.iterdir():
        if not pessoa_dir.is_dir():
            continue
        nome = pessoa_dir.name
        destino_pessoa = pasta_destino / nome
        destino_pessoa.mkdir(exist_ok=True, parents=True)

        for img_path in pessoa_dir.glob("*.*"):
            if img_path.suffix.lower() not in {".jpg",".jpeg",".png",".bmp",".webp"}:
                continue
            total_in += 1
            # Leitura com OpenCV (BGR)
            img_bgr = cv2.imread(str(img_path))
            if img_bgr is None:
                continue
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            # Detecta/alinha e retorna PIL Image 160x160
            face = mtcnn(img_rgb, save_path=None)
            if face is None:
                continue
            # Converte para numpy e salva
            face_np = face.permute(1,2,0).cpu().numpy()  # (160,160,3) RGB float[0..1]
            face_bgr = cv2.cvtColor((face_np*255).astype(np.uint8), cv2.COLOR_RGB2BGR)
            out_path = destino_pessoa / img_path.name
            cv2.imwrite(str(out_path), face_bgr)
            total_ok += 1

    print(f"Faces processadas: {total_ok}/{total_in} imagens")

processar_pasta_de_faces(PASTA_BRUTOS, PASTA_PROC)


FileNotFoundError: [Errno 2] No such file or directory: '/content/dados_brutos'


## 5) Carregar caminhos e fazer split (train/val/test)

Vamos listar as imagens processadas e separar em treino/validação/teste de forma estratificada.


In [ ]:

def listar_imagens_com_rotulos(pasta):
    paths = []
    labels = []
    for pessoa_dir in Path(pasta).iterdir():
        if not pessoa_dir.is_dir():
            continue
        label = pessoa_dir.name
        for img in pessoa_dir.glob("*.*"):
            if img.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".webp"}:
                paths.append(str(img))
                labels.append(label)
    return np.array(paths), np.array(labels)

X_paths, y_labels = listar_imagens_com_rotulos(PASTA_PROC)
print("Total imagens processadas:", len(X_paths))
print("Classes encontradas:", sorted(set(y_labels)))
assert len(set(y_labels)) == 4, "Certifique-se de ter exatamente 4 pastas (4 pessoas)!"

# Split train+val/test
X_tmp, X_test, y_tmp, y_test = train_test_split(
    X_paths, y_labels, test_size=0.2, random_state=SEED, stratify=y_labels
)
# Split train/val
X_train, X_val, y_train, y_val = train_test_split(
    X_tmp, y_tmp, test_size=0.2, random_state=SEED, stratify=y_tmp
)

len(X_train), len(X_val), len(X_test)


Total imagens processadas: 0
Classes encontradas: []


AssertionError: Certifique-se de ter exatamente 4 pastas (4 pessoas)!


## 6) Extrair embeddings (FaceNet)

Carregamos o `InceptionResnetV1` pré‑treinado (VGGFace2) e extraímos um vetor de **512 dimensões** por face.


In [ ]:

# Carrega FaceNet (pode baixar pesos na 1ª vez)
embedder = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def carregar_img_rgb(path):
    bgr = cv2.imread(path)
    if bgr is None:
        return None
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def img_to_embedding(img_rgb):
    # img_rgb: np.array (160,160,3) RGB uint8
    # Normaliza para [0,1] e para tensor (1,3,160,160)
    img_norm = (img_rgb.astype(np.float32) / 255.0)
    tensor = torch.from_numpy(img_norm).permute(2,0,1).unsqueeze(0).to(device)
    with torch.no_grad():
        emb = embedder(tensor).cpu().numpy()[0]  # (512,)
    return emb

def paths_to_embeddings(paths):
    E = []
    for p in paths:
        img = carregar_img_rgb(p)
        if img is None:
            # falha rara de leitura — cria embedding zero (ou pule)
            E.append(np.zeros(512, dtype=np.float32))
            continue
        emb = img_to_embedding(img)
        E.append(emb.astype(np.float32))
    return np.stack(E)

# Gera embeddings
E_train = paths_to_embeddings(X_train)
E_val   = paths_to_embeddings(X_val)
E_test  = paths_to_embeddings(X_test)

E_train.shape, E_val.shape, E_test.shape


  0%|          | 0.00/107M [00:00<?, ?B/s]

NameError: name 'X_train' is not defined

In [ ]:
print('/content/dados_processados/Luis-rm559100/df193e2c-ab94-4146-b7bd-f51058fe113f.JPG')

/content/dados_processados/Luis-rm559100/df193e2c-ab94-4146-b7bd-f51058fe113f.JPG



## 7) Treinar classificador (SVM RBF)

Usamos `SVC(C=10, kernel='rbf', probability=True)`. Sinta‑se livre para ajustar `C`.


In [ ]:

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc   = le.transform(y_val)
y_test_enc  = le.transform(y_test)

svm = SVC(C=10, kernel='rbf', gamma='scale', probability=True, random_state=SEED)
svm.fit(E_train, y_train_enc)

print("Acurácia (val):", svm.score(E_val, y_val_enc))
print("Acurácia (test):", svm.score(E_test, y_test_enc))


NameError: name 'y_train' is not defined


## 8) Métricas e Matriz de Confusão


In [ ]:

y_pred = svm.predict(E_test)
print(classification_report(y_test_enc, y_pred, target_names=le.classes_))

cm = confusion_matrix(y_test_enc, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
disp.plot(xticks_rotation=45)
plt.title("Matriz de Confusão — Teste")
plt.show()


NameError: name 'svm' is not defined


## 9) Salvar modelo e rótulos

Isto permite carregar rapidamente o classificador depois, sem precisar re-treinar.


In [ ]:

Path("modelos").mkdir(exist_ok=True)
joblib.dump(svm, "modelos/svm_faces.pkl")
joblib.dump(le,  "modelos/label_encoder.pkl")
print("Arquivos salvos em ./modelos/")



## 10) Inferência em uma imagem (exemplo)

Passe o caminho de uma imagem com rosto (de preferência **criada pelo passo de processamento**).
Se quiser passar uma imagem **bruta**, use a função `preprocessar_face_bruta` abaixo para alinhar antes.


In [ ]:

def prever_arquivo_imagem(path_img, threshold=0.60):
    # Carrega modelos
    clf = joblib.load("modelos/svm_faces.pkl")
    enc = joblib.load("modelos/label_encoder.pkl")

    img = carregar_img_rgb(path_img)
    if img is None:
        raise ValueError("Imagem não encontrada ou inválida.")
    emb = img_to_embedding(img)
    proba = clf.predict_proba([emb])[0]
    idx = int(proba.argmax())
    nome = enc.inverse_transform([idx])[0]
    conf = float(proba[idx])

    # threshold simples (opcional) para rejeitar "desconhecido"
    if conf < threshold:
        nome = "desconhecido"
    return nome, conf

# Exemplo de uso (ajuste o caminho):
# nome, conf = prever_arquivo_imagem("dados_processados/pessoa1_matheus/img001.jpg")
# print(nome, conf)



### (Opcional) Preprocessar rosto diretamente de uma imagem **bruta**

Se quiser fazer inferência em imagens que **não** passaram pelo passo de detecção/alinhamento, use esta função.


In [ ]:

def preprocessar_face_bruta(path_img_bruto):
    img_bgr = cv2.imread(str(path_img_bruto))
    if img_bgr is None:
        return None
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    face = mtcnn(img_rgb, save_path=None)
    if face is None:
        return None
    face_np = face.permute(1,2,0).cpu().numpy()  # (160,160,3) float [0..1]
    face_bgr = cv2.cvtColor((face_np*255).astype(np.uint8), cv2.COLOR_RGB2BGR)
    return face_bgr

def prever_imagem_bruta(path_img_bruto, threshold=0.60):
    face_bgr = preprocessar_face_bruta(path_img_bruto)
    if face_bgr is None:
        return "sem_face", 0.0
    # converter BGR->RGB e embutir
    img_rgb = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
    emb = img_to_embedding(img_rgb)
    clf = joblib.load("modelos/svm_faces.pkl")
    enc = joblib.load("modelos/label_encoder.pkl")
    proba = clf.predict_proba([emb])[0]
    idx = int(proba.argmax())
    nome = enc.inverse_transform([idx])[0]
    conf = float(proba[idx])
    if conf < threshold:
        nome = "desconhecido"
    return nome, conf


In [ ]:
prever_imagem_bruta('/content/IMG_3250.jpeg',threshold=0.60)

('sem_face', 0.0)


## 11) (Opcional) Webcam em tempo real

Detecta rosto com MTCNN, tira embedding e classifica com o SVM. **Pressione `q` para sair.**


In [ ]:

def webcam_demo(threshold=0.60):
    clf = joblib.load("modelos/svm_faces.pkl")
    enc = joblib.load("modelos/label_encoder.pkl")
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Não consegui abrir a webcam.")
        return
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            # OpenCV vem em BGR
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face = mtcnn(rgb)
            if face is not None:
                face_np = face.permute(1,2,0).cpu().numpy()
                face_rgb = (face_np*255).astype(np.uint8)
                emb = img_to_embedding(face_rgb)
                proba = clf.predict_proba([emb])[0]
                idx = int(proba.argmax())
                nome = enc.inverse_transform([idx])[0]
                conf = float(proba[idx])
                if conf < threshold:
                    nome = "desconhecido"
                texto = f"{nome} ({conf:.2f})"
                # desenhar retângulo/label simples
                cv2.putText(frame, texto, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow("Webcam - Reconhecimento Facial", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

# Para rodar: webcam_demo()


In [ ]:
webcam_demo()

FileNotFoundError: [Errno 2] No such file or directory: 'modelos/svm_faces.pkl'


## 12) Dicas de qualidade de dados

- Tenha **muitas amostras por pessoa** (ideal 50 a 200).
- Varie iluminação, ângulos, expressões e fundos.
- Evite óculos escuros/bonés em todas as fotos; inclua alguns casos, mas misture com fotos "limpas".
- Se um rosto não for detectado, ajuste o `margin` do MTCNN, melhore a qualidade ou filtre imagens ruins.
- Use `threshold` (0.60–0.80) para rejeitar desconhecidos.



---

# **Opção B (Alternativa):** Transfer Learning com Keras/MobileNetV2

Treina um **classificador com Softmax** diretamente na imagem recortada 160x160 a partir de um backbone `MobileNetV2`.
Funciona bem com dados suficientes; porém, para **datasets pequenos**, a abordagem de embeddings (FaceNet + SVM) tende a ser mais estável.


In [ ]:

# Rode somente se quiser testar a alternativa com Keras
# from tensorflow.keras import layers, models
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.optimizers import Adam
# import tensorflow as tf
# import glob

# # Carregar caminhos por classe
# def paths_labels_from_folder(pasta):
#     X, y = [], []
#     for pessoa in sorted(os.listdir(pasta)):
#         pdir = os.path.join(pasta, pessoa)
#         if not os.path.isdir(pdir):
#             continue
#         for img in glob.glob(os.path.join(pdir, "*.*")):
#             if os.path.splitext(img)[1].lower() in {".jpg",".jpeg",".png",".bmp",".webp"}:
#                 X.append(img); y.append(pessoa)
#     return np.array(X), np.array(y)

# X_all, y_all = paths_labels_from_folder(PASTA_PROC)
# X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, test_size=0.2, random_state=SEED, stratify=y_all)

# # tf.data para carregar e pre-processar
# def load_img_tf(path):
#     img = tf.io.read_file(path)
#     img = tf.io.decode_image(img, channels=3, expand_animations=False)
#     img = tf.image.resize(img, (160,160))
#     img = tf.cast(img, tf.float32)
#     img = preprocess_input(img)  # MobileNetV2
#     return img

# def gen_ds(X, y, batch=32, training=False):
#     ds = tf.data.Dataset.from_tensor_slices((X, y))
#     if training:
#         ds = ds.shuffle(buffer_size=len(X), seed=SEED)
#     ds = ds.map(lambda p, t: (load_img_tf(p), t), num_parallel_calls=tf.data.AUTOTUNE)
#     ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
#     return ds

# classes = sorted(set(y_all))
# le2 = LabelEncoder().fit(classes)
# y_tr_enc = le2.transform(y_tr)
# y_te_enc = le2.transform(y_te)

# ds_tr = gen_ds(X_tr, y_tr_enc, training=True)
# ds_te = gen_ds(X_te, y_te_enc, training=False)

# base = MobileNetV2(include_top=False, input_shape=(160,160,3), weights='imagenet')
# base.trainable = False  # comece congelado; opcionalmente descongele depois

# model = models.Sequential([
#     base,
#     layers.GlobalAveragePooling2D(),
#     layers.Dropout(0.3),
#     layers.Dense(len(classes), activation='softmax')
# ])

# model.compile(optimizer=Adam(1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.summary()

# hist = model.fit(ds_tr, validation_data=ds_te, epochs=10)
# model.evaluate(ds_te)

# # Descongelar parte do backbone (opcional)
# # base.trainable = True
# # for layer in base.layers[:-40]:
# #     layer.trainable = False
# # model.compile(optimizer=Adam(1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# # hist2 = model.fit(ds_tr, validation_data=ds_te, epochs=10)



---

### Pronto!
- Preencha `dados_brutos/` com as suas fotos e rode as células na ordem.
- Se quiser integrar ao seu `"[ Aula 17 ] - CKP04.ipynb"`, copie as seções que fizerem mais sentido.
- Se travar em algum erro específico, me mande o **stacktrace** que eu ajusto o notebook para você. :)
